# MCP Calculator Client with Mistral AI (STDIO Version)

This notebook demonstrates how to use MCP (Model Context Protocol) with Mistral AI's chat completion API using STDIO server connection.

## What is MCP?
Model Context Protocol (MCP) is a protocol that allows AI models to interact with external tools and services. In this example, we'll create a calculator service that can be called by Mistral AI.

## Prerequisites

Before running this notebook, make sure you have the required packages installed. Uncomment and run the cell below if needed:

In [ ]:
# Install required packages
# !pip install mcp mistralai

## Import Required Libraries

We need to import:
- `asyncio` for asynchronous operations
- `json` for parsing tool call arguments
- MCP client components for STDIO connection
- Mistral AI client for LLM interactions

In [ ]:
import asyncio
import json
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from mistralai import Mistral
import sys


## Configure Mistral AI Client

Replace the API key with your actual Mistral AI API key. You can get one from the Mistral AI platform.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
# Set your Mistral API key
if 'MISTRAL_KEY' not in os.environ:
    raise ValueError("MISTRAL_KEY environment variable is not set.")
mistral = Mistral(api_key=os.getenv('MISTRAL_KEY'))

## Initialize MCP Server Connection

This section establishes a connection to the MCP calculator server using STDIO (Standard Input/Output). The server should be running as a separate Python process that implements the calculator tools.

In [ ]:
# Configure server parameters for STDIO connection
server_params = StdioServerParameters(
    command=sys.executable,
    args=['2_mcp_calculator.py', "--server"]
)

# Connect to the MCP server and initialize session
async with stdio_client(server_params) as (read_stream, write_stream):
    async with ClientSession(read_stream, write_stream) as session:
        # Initialize session
        await session.initialize()
        
        # Get available tools from the server
        tools_result = await session.list_tools()
        print(f"\n🧮 Calculator ready! Found {len(tools_result.tools)} tools")

## Convert MCP Tools to Mistral Format

Mistral AI expects tools in a specific format. We need to transform the MCP tool definitions into Mistral's expected schema. Each tool needs:
- A `type` field set to "function"
- A `function` object containing the tool details
- Parameters defined as an object schema

In [ ]:
mistral_tools = []
for tool in tools_result.tools:
    tool_def = {
        "type": "function",  # Required wrapper for Mistral
        "function": {        # Function definition
            "name": tool.name,
            "description": tool.description,
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "number",
                        "description": "First number"
                    },
                    "b": {
                        "type": "number",
                        "description": "Second number"
                    }
                },
                "required": ["a", "b"]
            }
        }
    }
    mistral_tools.append(tool_def)

## Send a Calculation Request to Mistral

Now we'll ask Mistral to perform a calculation. The model will analyze the request and determine which tool to use (add or subtract).

In [ ]:
user_input = "Find 5 minus 2"

# Send the request to Mistral with tool definitions
response = mistral.chat.complete(
    model="mistral-small-latest",
    messages=[{
        "role": "system",
        "content": "You are a calculator assistant. Use the add or subtract functions to help the user with calculations."
    }, {
        "role": "user",
        "content": user_input
    }],
    tools=mistral_tools,
    tool_choice="auto"  # Let Mistral decide which tool to use
)

## Execute the Tool Call

Mistral will respond with a tool call request. We need to:
1. Extract the tool call from Mistral's response
2. Parse the arguments
3. Execute the tool using MCP
4. Return the result

In [ ]:
message = response.choices[0].message

# Re-establish connection to execute the tool
async with stdio_client(server_params) as (read_stream, write_stream):
    async with ClientSession(read_stream, write_stream) as session:
        # Initialize session
        await session.initialize()
        
        if message.tool_calls is not None:
            for tool_call in message.tool_calls:
                # Get function name from the tool call
                func_name = tool_call.function.name
                
                # Parse arguments (might be string or dict)
                if isinstance(tool_call.function.arguments, str):
                    args = json.loads(tool_call.function.arguments)
                else:
                    args = tool_call.function.arguments
                
                # Call the MCP tool and get the result
                result = await session.call_tool(func_name, args)
                print(f"Result: {result.content[0].text}")
        else:
            print("No tool calls found in the response.")

## Summary

This notebook demonstrated:
1. How to connect to an MCP server using STDIO
2. How to discover available tools from the server
3. How to convert MCP tool definitions to Mistral format
4. How to use Mistral AI to determine which tool to call
5. How to execute the tool call and get results

The STDIO approach is useful when you want to run the MCP server as a separate process, which provides better isolation and resource management.